In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Glucocorticoid_Sensitivity"
cohort = "GSE50012"

# Input paths
in_trait_dir = "../../input/GEO/Glucocorticoid_Sensitivity"
in_cohort_dir = "../../input/GEO/Glucocorticoid_Sensitivity/GSE50012"

# Output paths
out_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/GSE50012.csv"
out_gene_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/gene_data/GSE50012.csv"
out_clinical_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/clinical_data/GSE50012.csv"
json_path = "../../output/preprocess/Glucocorticoid_Sensitivity/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Comparison of cellular and transcriptional responses to 1,25-dihydroxyvitamin D3 and glucocorticoids in peripheral blood mononuclear cells"
!Series_summary	"Glucocorticoids (GC) and 1,25-dihydroxyvitamin D3 (1,25(OH)2 D3) are steroid hormones with anti-inflammatory properties with enhanced effects when combined. We previously showed that transcriptional response to GCs was correlated with inter-individual and inter-ethnic cellular response. Here, we profiled cellular and transcriptional responses to 1,25(OH)2 D3 from the same donors. We studied cellular response to combined treatment with GCs and 1,25(OH)2 D3 in a subset of individuals least responsive to GCs. We found that combination treatment had significantly greater inhibition of proliferation than with either steroid hormone alone. Overlapping differentially expressed (DE) genes between the two hormones were enriched for adaptive and innate immune processes. Non-overlapping differentially ex

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os

# 1. Gene Expression Data Availability
# Based on the series summary, it mentions transcriptional responses and RNA analysis,
# which suggests gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Trait row identification
# For Glucocorticoid_Sensitivity, looking at row 3 which has "in vitro lymphocyte gc sensitivity" 
# This is the appropriate measurement for our trait
trait_row = 3

# Age data is available in row 5
age_row = 5

# Gender data is available in row 4
gender_row = 4

# 2.2 Data Type Conversion functions
def convert_trait(value):
    """Convert glucocorticoid sensitivity values to continuous values."""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":")[1].strip()
    
    # Extract the numeric part from the gc sensitivity value
    if "in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex)" in value:
        try:
            return float(value.replace("in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex)", "").strip())
        except:
            return None
    
    try:
        return float(value)
    except:
        return None

def convert_age(value):
    """Convert age values to continuous values."""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":")[1].strip()
    
    # Extract the numeric part from the age value
    if "age (years)" in value:
        try:
            return float(value.replace("age (years)", "").strip())
        except:
            return None
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender values to binary (0 for female, 1 for male)."""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":")[1].strip()
    
    # Convert gender to binary
    value = value.lower()
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial filtering and saving metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Check if clinical data is available (trait_row is not None)
if trait_row is not None:
    try:
        # Extract clinical features from the clinical_data DataFrame which should be available in the environment
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,  # Use the clinical_data variable that should be available
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Create the output directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the dataframe to CSV
        selected_clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical features: {e}")
else:
    print("No clinical data available (trait_row is None)")


Preview of selected clinical features:
{'GSM832137': [89.43486, nan, nan], 'GSM832138': [89.43486, nan, nan], 'GSM832139': [89.43486, nan, nan], 'GSM832140': [89.43486, nan, nan], 'GSM832141': [95.88507, nan, nan], 'GSM832142': [95.88507, nan, nan], 'GSM832143': [95.88507, nan, nan], 'GSM832144': [95.88507, nan, nan], 'GSM832145': [95.22036, nan, nan], 'GSM832146': [95.22036, nan, nan], 'GSM832147': [95.22036, nan, nan], 'GSM832148': [95.22036, nan, nan], 'GSM832149': [92.86704, nan, nan], 'GSM832150': [92.86704, nan, nan], 'GSM832151': [92.86704, nan, nan], 'GSM832152': [92.86704, nan, nan], 'GSM832153': [93.71633, nan, nan], 'GSM832154': [93.71633, nan, nan], 'GSM832155': [93.71633, nan, nan], 'GSM832156': [93.71633, nan, nan], 'GSM832157': [96.76962, nan, nan], 'GSM832158': [96.76962, nan, nan], 'GSM832159': [96.76962, nan, nan], 'GSM832160': [96.76962, nan, nan], 'GSM832161': [88.55031, nan, nan], 'GSM832162': [88.55031, nan, nan], 'GSM832163': [88.55031, nan, nan], 'GSM832164': [8

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 70
Header line: "ID_REF"	"GSM832137"	"GSM832138"	"GSM832139"	"GSM832140"	"GSM832141"	"GSM832142"	"GSM832143"	"GSM832144"	"GSM832145"	"GSM832146"	"GSM832147"	"GSM832148"	"GSM832149"	"GSM832150"	"GSM832151"	"GSM832152"	"GSM832153"	"GSM832154"	"GSM832155"	"GSM832156"	"GSM832157"	"GSM832158"	"GSM832159"	"GSM832160"	"GSM832161"	"GSM832162"	"GSM832163"	"GSM832164"	"GSM832165"	"GSM832166"	"GSM832167"	"GSM832168"	"GSM832169"	"GSM832170"	"GSM832171"	"GSM832172"	"GSM832173"	"GSM832174"	"GSM832175"	"GSM832176"	"GSM832177"	"GSM832178"	"GSM832179"	"GSM832180"	"GSM832181"	"GSM832182"	"GSM832183"	"GSM832184"	"GSM1212354"	"GSM1212355"	"GSM1212356"	"GSM1212357"	"GSM1212358"	"GSM1212359"	"GSM1212360"	"GSM1212361"	"GSM1212362"	"GSM1212363"	"GSM1212364"	"GSM1212365"	"GSM1212366"	"GSM1212367"	"GSM1212368"	"GSM1212369"	"GSM1212370"	"GSM1212371"	"GSM1212372"	"GSM1212373"	"GSM1212374"	"GSM1212375"	"GSM1212376"	"GSM1212377"
First data line: "ILMN_1343291"	14.12073024	14.1847953	14.327

Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These are Illumina probe IDs (starting with "ILMN_"), not human gene symbols
# Illumina probe IDs need to be mapped to official gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE50012
Line 6: !Series_title = Comparison of cellular and transcriptional responses to 1,25-dihydroxyvitamin D3 and glucocorticoids in peripheral blood mononuclear cells
Line 7: !Series_geo_accession = GSE50012
Line 8: !Series_status = Public on Aug 21 2013
Line 9: !Series_submission_date = Aug 20 2013
Line 10: !Series_last_update_date = Aug 13 2018
Line 11: !Series_pubmed_id = 24116131
Line 12: !Series_pubmed_id = 24550213
Line 13: !Series_summary = Glucocorticoids (GC) and 1,25-dihydroxyvitamin D3 (1,25(OH)2 D3) are steroid hormones with anti-inflammatory properties with enhanced effects when combined. We previously showed that transcriptional response to GCs was correlated with inter-individual and int


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180, 6510136, 7560739, 1450438, 1240647], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCATGTGATACGA

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the keys for gene identifiers and gene symbols in the gene annotation
# From the preview, we can see:
# - 'ID' column contains the Illumina probe IDs (starting with ILMN_)
# - 'Symbol' column contains the gene symbols

# 2. Get the gene mapping dataframe
prob_col = 'ID'  # Column containing probe IDs
gene_col = 'Symbol'  # Column containing gene symbols

# Extract the mapping between probe IDs and gene symbols
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Gene mapping dataframe - first 5 rows:")
print(mapping_df.head())
print(f"Total number of mappings: {len(mapping_df)}")

# 3. Convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print(f"First 5 gene symbols after mapping:")
print(gene_data.index[:5])

# Save the gene expression data to the output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe - first 5 rows:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB
Total number of mappings: 44837
Gene expression data shape after mapping: (21372, 72)
First 5 gene symbols after mapping:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1'], dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Glucocorticoid_Sensitivity/gene_data/GSE50012.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data
clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
print(f"Clinical data shape: {clinical_data.shape}")
print("Clinical data preview:")
print(clinical_data.head())

# 3. Link clinical and genetic data using proper sample IDs
# First, transpose gene expression data to have samples as rows
gene_data_t = normalized_gene_data.T

# The clinical data should have samples as columns - verify sample IDs match
gene_samples = set(gene_data_t.index)
clinical_samples = set(clinical_data.columns)
common_samples = gene_samples.intersection(clinical_samples)

print(f"Gene samples: {len(gene_samples)}")
print(f"Clinical samples: {len(clinical_samples)}")
print(f"Common samples: {len(common_samples)}")

# Use the geo_link_clinical_genetic_data function to properly link the data
linked_data = geo_link_clinical_genetic_data(clinical_data, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
if linked_data.shape[1] >= 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data.head())

# 4. Handle missing values
print("\nMissing values before handling:")
print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
if gene_cols:
    missing_genes_pct = linked_data[gene_cols].isna().mean()
    genes_with_high_missing = sum(missing_genes_pct > 0.2)
    print(f"  Genes with >20% missing: {genes_with_high_missing}")
    
    if len(linked_data) > 0:  # Ensure we have samples before checking
        missing_per_sample = linked_data[gene_cols].isna().mean(axis=1)
        samples_with_high_missing = sum(missing_per_sample > 0.05)
        print(f"  Samples with >5% missing genes: {samples_with_high_missing}")

# Handle missing values
cleaned_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 5. Evaluate bias in trait and demographic features
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)

# 6. Final validation and save
note = "Dataset contains gene expression data from glucocorticoid sensitivity studies. "
if 'Age' in cleaned_data.columns:
    note += "Age data is available. "
if 'Gender' in cleaned_data.columns:
    note += "Gender data is available. "

is_gene_available = len(normalized_gene_data) > 0
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=True, 
    is_biased=trait_biased, 
    df=cleaned_data,
    note=note
)

# 7. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")

Gene data shape after normalization: (20259, 72)
Sample gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS']


Gene data saved to ../../output/preprocess/Glucocorticoid_Sensitivity/gene_data/GSE50012.csv
Clinical data shape: (3, 72)
Clinical data preview:
                            GSM832137  GSM832138  GSM832139  GSM832140  \
Glucocorticoid_Sensitivity   89.43486   89.43486   89.43486   89.43486   
Age                               NaN        NaN        NaN        NaN   
Gender                            NaN        NaN        NaN        NaN   

                            GSM832141  GSM832142  GSM832143  GSM832144  \
Glucocorticoid_Sensitivity   95.88507   95.88507   95.88507   95.88507   
Age                               NaN        NaN        NaN        NaN   
Gender                            NaN        NaN        NaN        NaN   

                            GSM832145  GSM832146  ...  GSM1212368  GSM1212369  \
Glucocorticoid_Sensitivity   95.22036   95.22036  ...       24.00        8.00   
Age                               NaN        NaN  ...       21.56       21.56   
Gender            

Data shape after handling missing values: (72, 20262)
Quartiles for 'Glucocorticoid_Sensitivity':
  25%: 24.0
  50% (Median): 89.767215
  75%: 94.17097
Min: 8.0
Max: 98.34904
The distribution of the feature 'Glucocorticoid_Sensitivity' in this dataset is fine.

Quartiles for 'Age':
  25%: 25.745
  50% (Median): 25.745
  75%: 25.745
Min: 20.38
Max: 44.15
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 12 occurrences. This represents 16.67% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Glucocorticoid_Sensitivity/GSE50012.csv
